# Esri ZIP Codes with race/ethnicity

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import json
import jenkspy
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
alt.data_transformers.disable_max_rows()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
### Get clean Esri data provided to the Times by the company's press office

### Fetch our ZIP Codes file with race/ethnicity demographics

In [4]:
# From: https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/USA_ZIP_Codes/FeatureServer/0

In [17]:
ca_zips_race = gpd.read_file('/Users/mhustiles/data/gis/ca-zip-codes-esri-demographics.geojson')

In [18]:
ca_zips_race.columns = ca_zips_src.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

/var/folders/hm/zcc9d9vn2q15_wh0_0k06ckw0000gn/T/ipykernel_86579/1825410642.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ca_zips_race.columns = ca_zips_src.columns.str.strip().str.lower().str.replace(' ', '_')\


In [23]:
ca_zips_race[['objectid', 'zip', 'po_name', 'pop2012', 'white', 'black', 'ameri_es',
       'asian', 'hawn_pi', 'hispanic', 'other', 'mult_race', 'med_age',
       'ave_hh_sz', 'marhh_chd', 'families', 'ave_fam_sz', 'hse_units', 'sqmi']].to_csv('data/ca-zip-codes-esri-demographics.csv', index=False)

### Slim down the geodataframe and clean up column names

In [21]:
ca_zips_race = gpd.GeoDataFrame(ca_zips_race[[ 'zip', 'po_name', 'population_nu', 'white','black','ameri_es','asian','hawn_pi','hispanic','other','mult_race','geometry']])

KeyError: "['population_nu'] not in index"

In [ ]:
ca_zips_race.rename(columns={ 'po_name':'location', 'population_nu':'population'}, inplace=True)

### Calculate rates for race/ethnicity by ZIP Codes

In [ ]:
# ca_zips_race['white_alone'] = ca_zips_race['hispanic'] - ca_zips_race['white']

In [ ]:
# ca_zips_race['white_pct'] = round((ca_zips_race['white_alone'] / ca_zips_race['population'])*100, 2)

In [ ]:
ca_zips_race['black_pct'] = round((ca_zips_race['black'] / ca_zips_race['population'])*100, 2)

In [88]:
ca_zips_race['asian_pct'] = round((ca_zips_race['asian'] / ca_zips_race['population'])*100, 2)

In [89]:
ca_zips_race['hispanic_pct'] = round((ca_zips_race['hispanic'] / ca_zips_race['population'])*100, 2)

In [91]:
ca_zips_race.sort_values('hispanic_pct', ascending=False).head(10)

,zip,location,population,white,black,ameri_es,asian,hawn_pi,hispanic,other,mult_race,geometry,black_pct,asian_pct,hispanic_pct
17676,90023,Los Angeles,43866,21780,324,550,255,31,42765,19522,1404,"POLYGON ((-118.22804 34.03852, -118.22707 34.0...",0.74,0.58,97.49
18161,90270,Maywood,25842,13424,159,197,83,17,25185,10843,1118,"POLYGON ((-118.20428 33.98808, -118.20421 33.9...",0.62,0.32,97.46
18796,92254,Mecca,12803,4331,71,81,41,12,12471,7895,372,"POLYGON ((-116.05959 33.61375, -116.05958 33.6...",0.55,0.32,97.41
17475,90255,Huntington Park,76772,40224,524,1069,490,30,74603,31604,2832,"POLYGON ((-118.23790 33.98912, -118.23767 33.9...",0.68,0.64,97.17
18129,90063,Los Angeles,61362,29774,467,717,766,28,59365,27544,2065,"POLYGON ((-118.19269 34.06371, -118.19268 34.0...",0.76,1.25,96.75
17675,90022,Los Angeles,65157,33861,364,820,720,36,63023,27179,2175,"POLYGON ((-118.16849 34.04848, -118.16851 34.0...",0.56,1.11,96.72
18696,92231,Calexico,39005,23440,136,206,510,22,37708,13028,1663,"POLYGON ((-115.53408 32.70905, -115.53408 32.7...",0.35,1.31,96.67
17337,93234,Huron,7038,2361,68,77,37,6,6790,4180,308,"POLYGON ((-120.01420 36.56107, -120.02096 36.5...",0.97,0.53,96.48
18775,92249,Heber,6507,3688,35,44,39,0,6197,2320,380,"POLYGON ((-115.53408 32.70905, -115.53172 32.7...",0.54,0.60,95.24
17434,90201,Bell Gardens,98179,49945,896,1008,624,66,93354,41856,3783,"POLYGON ((-118.17405 33.99873, -118.17473 33.9...",0.91,0.64,95.09


In [98]:
ca_zips_race['hispanic'].mean()

8210.019366197183

### Export this more manageable file

In [93]:
ca_zips_race.to_file('../../cars/_notebooks/data/processed/ca_zips_race.geojson', driver='GeoJSON')